In [ ]:
from tinyRAG.embedding import VectorStore,read_file_content,MyEmbedding #  我写的RAG，在同个文件夹下

In [ ]:
data  = []
with open("D:\code\competiton\饿了么智慧点餐\RAG的模型测试\dim_ai_exam_food_category_filter_out.txt",'r',encoding = 'utf-8') as f:
    for line in f:
        
        data.append('[SEP]'.join(line.strip().split('\t')))

data = data[1:]
data

In [ ]:
model = 'google-bert/bert-base-chinese'
embedding_model = MyEmbedding(path=model)


In [ ]:
# vs = VectorStore(data)
# vs.get_vector(EmbeddingModel=embedding_model)
# vs.persist(path = "bert数据库")
vs  = VectorStore()
vs.load_vector( path = r'bert数据库')
# 查询示例
query = '鱼豆腐汤'
result = vs.query(query = query,EmbeddingModel=embedding_model,k=10)
result

In [ ]:
import json
with open('D:\code\competiton\饿了么智慧点餐\RAG的模型测试\one_text_ds.json','r',encoding="utf-8") as f:
    text = json.load(f)

In [ ]:
text

In [ ]:
ans = []
from tqdm import tqdm
for i in tqdm(text):
    ans.append(vs.query(i,EmbeddingModel=embedding_model,k=1))

In [ ]:
ans

In [ ]:
with open('bert结果.txt',"w",encoding="utf-8") as f:
    for i in range(len(ans)):
        f.write(ans[i][0].split('[SEP]')[0] + '\t' + text[i]+ '\n')

In [ ]:
from openai import OpenAI
client = OpenAI(api_key='sk-30d9e5ff6f22478687ae7d87f6615edc', base_url="https://api.deepseek.com")

In [ ]:
def dummy_response_R1(example,client):
        messages = []
        messages.append({"role": "system", 
        "content": 
        f''' 
        角色：你是一个对RAG召回结果判断是否符合的判别大师
        任务：判断召回结果和原词条是否相符
        生成条件：只能生成1条文本，你要理解样例，并类比样例对召回结果和原词条进行判断，生成结果有三种：1.召回结果符合原词条,召回结果的范围可以比原词条的大一些，口味可以不一致，；2.召回结果部分满足原词条，即召回结果的范围比原词条小；3.召回结果和原词条完全不相符。
        禁止：禁止生成重复的内容和问答句子，禁止生成无关内容；输出的文本中不能包含任何多余的内容。禁止输出非json格式的内容。
        工作流：参考以下条样例，扮演好你的角色，生成对应的json格式的输出
        注意：召回结果会有商家起的更可爱的名字，如皮蛋 - 皮皮蛋之类的，你要正确的识别出来
        样例： 
        1.输入： 原词条：番茄鸡蛋盖饭；召回结果：番茄炒鸡蛋；
          输出： {{"判断相符":'2','原因':"召回结果部分满足原词条，原词条的是盖饭，召回结果是没有饭，" }}
        2.输入： 原词条：番茄蛋花面疙瘩汤；召回结果：番茄豆腐鸡蛋疙瘩汤；
          输出：{{"判断相符":'1','原因':"召回结果符合原词条，召回结果范围比原词条大，但是可以覆盖原词条" }}
        3.输入：原词条：炒葱香干豆腐丝；召回结果：豆腐皮裹香菜；
          输出：{{"判断相符":'2','原因':"召回结果部分满足原词条，原词条是抄葱，召回结果是香菜" }}
        4.输入：原词条：冬瓜排骨汤泡饭；召回结果：排骨小青菜泡饭；
          输出：{{"判断相符":'2','原因':"召回结果部分满足原词条，召回结果少了冬瓜" }}
        5.输入：原词条：粥热乎的；召回结果：暖暖的小米粥；
          输入：{{"判断相符":'1','原因':"召回结果符合原词条" }}
        6.输入：原词条：土鸡汤；召回结果：茶树菇土鸡汤；
          输出：{{"判断相符":'1','原因':"召回结果符合原词条，召回结果比原词条大，但是和原词条相符" }}
        7.输入：原词条：姜汤；召回结果：牛肉当归粥；
          输出：{{"判断相符":'3','原因':"召回结果和原词条完全不相符" }}
        8.输入：原词条：鲜鱼嫩豆腐汤；召回结果：海鲜嫩豆腐汤；
          输出：{{"判断相符":'2','原因':"召回结果和原词条部分相符" }}
        以下是你要改写的内容：
        原词条：{example[0]}；召回结果：{example[1]}
        不需要其他的内容。请注意，输出的文本中不能包含任何多余的内容，输出为json格式，输出的json的"原因"要加上你的理解的部分。
        ''' })

        response = client.chat.completions.create(
        model="deepseek-chat",
        messages=messages,
        response_format={
        'type': 'json_object'
        }
          )
        response =  response.choices[0].message.content
        print(response+"\n"+example[0]+'\t'+example[1])
        return response

In [ ]:
ans,len(ans)

In [ ]:
text,len(text)

In [ ]:
text[1],ans[1][0].split('[SEP]')[0]

In [41]:
responses = []
for i in tqdm(range(len(ans))):
    example = [ text[i],ans[i][0].split('[SEP]')[0]]
    response = dummy_response_R1 (example,client)
    responses.append(response)


 93%|█████████▎| 1845/1978 [16:49:21<14:13,  6.42s/it]

{"判断相符":'2','原因':"召回结果部分满足原词条，召回结果比原词条多了红豆" }
黑米粥	黑米红豆粥


 93%|█████████▎| 1846/1978 [16:49:26<13:48,  6.27s/it]

{"判断相符":'3','原因':"召回结果和原词条完全不相符，原词条是豆芽，召回结果是土豆丝" }
家常小炒嫩豆芽	家常小炒土豆丝


 93%|█████████▎| 1847/1978 [16:49:32<13:02,  5.97s/it]

...

{"判断相符":'1','原因':"召回结果符合原词条，召回结果比原词条详细，但核心成分山药和肉都包含在内" }
山药肉粥	山药瘦肉砂锅粥


100%|█████████▉| 1977/1978 [17:03:34<00:05,  5.39s/it]

{"判断相符":'1','原因':"召回结果符合原词条，召回结果包含了原词条的内容" }
韭黄炒蛋	韭菜炒蛋韭黄炒蛋


100%|██████████| 1978/1978 [17:03:41<00:00, 31.05s/it]

{"判断相符":'1','原因':"召回结果完全符合原词条" }
香菇青菜肉末粥	香菇青菜肉末粥


In [45]:
responses

['{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条多加了肉末，但基本覆盖原词条" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果范围比原词条大，但是可以覆盖原词条" }',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，原词条是香辣，召回结果是蒜蓉，口味不一致" }',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，原词条的是拌面，召回结果是没有面，" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条多加了红糖，但基本成分一致" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条大，但是和原词条相符" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果使用了更可爱的名字\'皮皮蛋\'，但实质内容与原词条一致" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果范围比原词条大，增加了黄瓜片，但可以覆盖原词条" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果和原词条相符，只是表述方式不同" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果与原词条描述一致，只是表述方式略有不同"}',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，原词条是酸辣白菜，召回结果是老醋里脊白菜，口味和主料部分相符但增加了里脊" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条大，但是和原词条相符" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果使用了更可爱的名字\'皮皮蛋\'，但实质内容与原词条一致"}',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果只是商家起的更可爱的名字，内容与原词条一致" }',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，召回结果多了糯米" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条大，但是和原词条相符" }

In [46]:
with open ('result.txt','w',encoding='utf-8') as f:
    for line in responses:
        f.write(line+'\n')

In [1]:
with open(r'D:\code\competiton\饿了么智慧点餐\RAG的模型测试\bert\result.txt','r',encoding= 'utf-8') as f:
    data = []
    for line in f:
        data.append(line.strip())
        

In [2]:
data

['{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条多加了肉末，但基本覆盖原词条" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果范围比原词条大，但是可以覆盖原词条" }',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，原词条是香辣，召回结果是蒜蓉，口味不一致" }',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，原词条的是拌面，召回结果是没有面，" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条多加了红糖，但基本成分一致" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条大，但是和原词条相符" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果使用了更可爱的名字\'皮皮蛋\'，但实质内容与原词条一致" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果范围比原词条大，增加了黄瓜片，但可以覆盖原词条" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果和原词条相符，只是表述方式不同" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果与原词条描述一致，只是表述方式略有不同"}',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，原词条是酸辣白菜，召回结果是老醋里脊白菜，口味和主料部分相符但增加了里脊" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条大，但是和原词条相符" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果使用了更可爱的名字\'皮皮蛋\'，但实质内容与原词条一致"}',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果只是商家起的更可爱的名字，内容与原词条一致" }',
 '{"判断相符":\'2\',\'原因\':"召回结果部分满足原词条，召回结果多了糯米" }',
 '{"判断相符":\'1\',\'原因\':"召回结果符合原词条，召回结果比原词条大，但是和原词条相符" }

In [3]:
one = 0
two = 0
three = 0

for line in data:
    if '1' in line:
        one += 1
    if '2' in line:
        two += 1
    if '3' in line:
        three += 1
         

In [4]:
one,two,three

(1126, 518, 334)

In [5]:
print((one+two)/(one+two+three))

0.8311425682507584
